# *$$ \mathrm{Modelo \ VC-HBO \ para \ el \ consumidor \ Postkeynesiano} $$*
Oscar Hurtado González, Luis Omar Barbosa García

# Introducción y Objetivos

## Modelo del consumidor Postkeynessiano

* Teoría realista que integra elementos distintos a la teoría del consumidor Neoclásica

* Su estructura permite ser estudiada desde la perspectiva de los Sistemas Complejos 

Algunos de sus elementos básicos:

* Preferencias lexicográficas 

* Crecimiento y consumo por imitación 

* Principio de dependencia y el consumo por moda

* Herencia 

## Modelo Vite-Carreón basado en agentes

En este modelo, el consumo se modela por un vector  

$$ V_c = (x,y,z)$$

Se clasifica la clase social de acuerdo a el valor de las entradas  $ x,y,z$.

El **consumo por moda** (principio de dependencia) se modela a partir de reglas dependientes en la vecindad de Moore a la que pertenecen. Estas reglas aparecen en la Tabla 2 de [Vite-Carreón].

EL **consumo por imitación** se implementa mediante un modelo probabilístico: ![](proba.jpg)

Figura 3. Tomada de [Vite-Carreón]

## Modelo VC-HBO, objetivo

* El *objetivo general* es seguir estudiando al consumidor Postkeynesiano desde la perspectiva del modelado basado en agentes 


* El *objetivo específico* de nuestra propuesta es modelar las practicas de consumo por imitación no de forma probabilístican, sino a partir de la idea "mi consumo, es el ingreso de otros"

# Metodología

Se añade a los agentes un atributo de clase social $(c_a)$ y un atributo de  riqueza/ingreso ($w_a$). 

**Inicialización del modelo:** Número de agentes en cada clase social con: $w_a = x_b$, a los de media $w_a = x_m$  ya los de alta $w_a = x_a$. Asignación de la riqueza inicial de forma probabilística:

* $0\leq x_b < U_M$ 


* $U_M < x_m \leq U_A$


* $U_A \leq x_a < max$

La clase social asignada, dependerá de la riqueza poseída. Esto, en contraste con [Vite-Carreón] en donde la clase social estaba determinada exclusivamente con las prácticas de consumo codificadas en el vector de consumo $V_c$.  Las reglas para asignar la clase social son las siguientes:

**Condiciones de pertenencia de  clase**

$$  \mathrm{Si \ } w_a \leq U_B \Rightarrow \mathrm{el \ agente \ es \ de \ clase \ baja \ } ( c_a = -1 )$$

$$  \mathrm{Si \ } U_B  \leq w_a \leq U_A \Rightarrow \mathrm{el \ agente \ es \ de \ clase \ media \ } ( c_a = 0 )$$

$$  \mathrm{Si \ } U_A \leq w_a \Rightarrow \mathrm{el \ agente \ es \ de \ clase \ alta \ } ( c_a = 1 ),$$

Al igual que en [Vite-Carreon], se asigna también un vector de consumo $ V_c \in {(1,0,0), (0,1,0), (0,0,1)} $, en función de la clase inicial.



**Hipótesis importantes**

* Todos los agentes son productores pero producen bienes exclusivamente dentro de su clase social. La producción de cada agente está determinada por $c_a$. 


* Todos los agentes tienen la capacidad de suplir la oferta total de sus bienes. Tal oferta esta determinada por los agentes que lo ``rodean''. 


* El agente consume un solo bien a lo más de cada categoría y que los precios entre categorías son diferentes: los bienes de clase baja cuestan una unidad de riqueza, los bienes de clase media dos unidades y los de clase alta tres unidades

**Regla básica de consumo:** 

* El agente evalúa su riqueza y su vector de consumo, busca a productores *en su vecindad de Moore*, escoge a quienes comprarles, evalúa su riqueza  después de que consumió y determina su clase social. Después, utilizando las reglas de actualización del vector de consumo definidas en [Vite-Carreón], cambia su vector de consumo de ser necesario. 


* Es importante mencionar que el *orden de consumo* el **orden lexicográfico** discutido en la teoría del consumidor Postkeynesiano.

## Code

In [1]:
from mesa import Agent, Model
from mesa.space import SingleGrid
from mesa.time import RandomActivation, SimultaneousActivation, BaseScheduler
from mesa.datacollection import DataCollector
from mesa.visualization.modules import ChartModule
import numpy as np
import matplotlib.pyplot as plt
import random as rm

from mesa.visualization.modules import CanvasGrid
from mesa.visualization.ModularVisualization import ModularServer
from termcolor import colored


In [2]:
class KeynesAgent(Agent):
    """
         Create a new PostKeynesian agent.
         Args:
            model: class model in wich the agent lives
            pos: Posicion of the agent (tuple or list?)
            v_c: Consume vector: informacion about the consuming behavior of the agent
                    (tuple, list, or numpy vector??)
        """
    D=dict(B=np.array([1,0,0]), M=np.array([0,1,0]), A=np.array([0,0,1]),
           MB=np.array([1,1,0]), AB=np.array([1,0,1]), AM=np.array([0,1,1]),
           AMB=np.array([1,1,1]))
    
    
    
#     # Clases sociales 
    
    D_med=[D['M'], D['MB'], D['AM']] # media: reglas 2,3,4,5
    D_alt=[D['A'], D['AB'], D['AMB']] # alta: reglas 2,3,4,5

#----------------------init method-------------------------------------------
    
    def __init__(self, model, pos, v_c, wealth):
        #self.model=model
        super().__init__(pos, model)
        self.pos = pos
        self.v_c = v_c
        self.clase = None # será = -1 si el agente es pobre, 0 si medio, 1 si es alta
        self.wealth = wealth
        
        self.coin = None
        self.vecindad = None
        self.suma_vecinos = None
        
        self.contador = 0
        
        


#------------------------------CONSUMO POR MODA--------------------------------------- 

    #reglas de actualización para clase baja
    def med_rules(self):
        s = self.suma_vecinos
        if s[1] > s[0] and s[1] > s[2]: #1 medio 
            self.v_c = self.D['M']
            # self.clase = 0

        elif s[0] < s[2] and s[1] < s[2]: #2 medio
            self.v_c = self.D['AM']
            # self.clase = 0

        elif s[2] < s[0] and s[1] < s[0]: #3 medio
            self.v_c = self.D['MB']
            # self.clase =0

        elif s[0] == s[1]:
            self.v_c = self.D['MB']
            # self.clase = 0

        elif s[1] == s[2]:
            self.v_c = self.D['AM']
            # self.clase = 0

        elif s[0] == s[2]: #4 medio 
            self.v_c = self.D['AMB']
            # self.clase = 1

    #reglas de actualización para clase alta
    def alt_rules(self):
        s= self.suma_vecinos
        if s[2] > s[0] and s[2] > s[1]: #5 alto
            self.v_c = self.D['A']
            # self.clase = 1

        elif s[1] > s[0] and s[1] > s[2]: #6 alto
            self.v_c = self.D['AM']
            # self.clase = 0

        elif s[1] < s[0] and s[2] < s[0]: #7 alto
            self.v_c = self.D['AB']
            # self.clase = 1

        elif s[0] == s[1]: #8 alto
            self.v_c = self.D['AMB']
            # self.clase = 1

        elif s[1] == s[2]:
            self.v_c = self.D['AM']
            # self.clase = 0

        elif s[0] == s[2]:
            self.v_c =self.D['AB']
            # self.clase = 1

    def vc_update(self):
        
        if self.clase == -1:
            
            # self.vecindad = np.array([neig.v_c for neig in model.grid.get_neighbors(self.pos,moore=True)])
            # self.suma_vecinos = self.vecindad.sum(0)
            #print(self.suma_vecinos)
            self.v_c = self.D['B']
            # self.clase = -1
            
        
        elif self.clase == 0:

            self.vecindad = np.array([neig.v_c for neig in model.grid.get_neighbors(self.pos,moore=True)])
            self.suma_vecinos = self.vecindad.sum(0)
            #print(self.suma_vecinos)
            self.med_rules()
           
        elif self.clase == 1:
            
            self.vecindad = np.array([neig.v_c for neig in model.grid.get_neighbors(self.pos,moore=True)])
            self.suma_vecinos = self.vecindad.sum(0)
            #print(self.suma_vecinos)
            self.alt_rules()
            

            
#----------------------------------------------------------------------------------
            
#----------------------------CONSUMO POR IMITACION---------------------------------
#modelo probabilistico de transito entre clases
                  
    def proba(self):

        D=dict(B=np.array([1,0,0]), M=np.array([0,1,0]), A=np.array([0,0,1]),
           MB=np.array([1,1,0]), AB=np.array([1,0,1]), AM=np.array([0,1,1]),
           AMB=np.array([1,1,1]))
        
        D_med=[D['M'], D['MB'], D['AM']] # media: reglas 2,3,4,5
        D_alt=[D['A'], D['AB'], D['AMB']] # alta: reglas 2,3,4,5
                  
        if self.clase == -1:

            coin = rm.choices([0,1], weights=[0.9,0.1] )[0]
            self.coin=coin
            if coin == 1:
                self.v_c = rm.choice(D_med)
                self.clase = 0
                

        elif self.clase == 0:

            coin3 = rm.choices([-1,0,1], weights=[0.1,0.8,0.1] )[0]
            self.coin=coin3
            #print(coin3)
            if coin3 == -1:
                self.v_c = D['B']
                self.clase = -1
            elif coin3 == 1:
                self.v_c = rm.choice(D_alt)
                self.clase = 1
                           
                
        elif self.clase == 1:
            coin = rm.choices([0,1], weights=[0.9,0.1] )[0]
            self.coin=coin
            if coin == 1:
                self.v_c = rm.choice(D_med)
                self.clase = 0
       
      
        #print(self.clase)
    
#---------------------------CONSUMO POR IMITACION------------------------------------------
#no probabilistico

    def select_producer(self,index):
        
        vecindad = self.model.grid.get_neighbors(self.pos, moore=True)
        #print([j.clase for j in vecindad])
        producers = []
        
        for a in vecindad: #puedo hacer esto en un list comprhension con condicional
            if a.clase == (index-1):
                producers.append(a)
        if len(producers) > 0:
            #print(producers)        
            lucky_prod = rm.choice(producers)
            lucky_prod.wealth += (index+1)
            self.wealth -= (index+1)
        
        #perdonar deuda y actualizar clase social
        
#         if self.wealth < 0:
#             self.wealth = 0
#             self.clase = -1

    def select_producer_2(self):
        #print(f"Clase inicial:{self.clase}")
        vecindad = self.model.grid.get_neighbors(self.pos, moore=True)        
        producers = [vecino for vecino in vecindad if self.clase == vecino.clase]
        if producers:
            monto = 0
            if self.clase == -1:
                monto = 1
            elif self.clase == 0:
                monto = 2
            else:
                monto = 3
            lucky_prod = rm.choice(producers)
            lucky_prod.wealth += monto
            self.wealth -= monto

            
    def agent_spend(self):
        
#         if self.wealth <0:
#             #no gasta
#             self.wealth=self.wealth
            
        if 0 <= self.wealth <= self.model.umbral_M and self.v_c[0] == 1: 
            #print('voy a gastar como pobre')
            # self.select_producer(0)
            self.select_producer_2()
        
        if self.model.umbral_M < self.wealth <= self.model.umbral_A and self.v_c[1] == 1:
            #print('voy a gastar como medio')
            # self.select_producer(1)
            self.select_producer_2()
            
        
        if self.wealth > self.model.umbral_A and self.v_c[2] == 1:
            #print('voy a gastar como rico')
            # self.select_producer(2)
            self.select_producer_2()
            
        #balance
        #self.balance()
        if self.wealth < 0:
        #    self.wealth = 0 # Se perdonan las deudas
           self.clase = -1
        elif 0 <= self.wealth <= self.model.umbral_M:
            self.clase = -1
        elif self.model.umbral_M < self.wealth <= self.model.umbral_A:
            self.clase = 0
        else:
            self.clase = 1
        #print(f'clase actualizada: {self.clase}, vc: {self.v_c}, wealth: {self.wealth}, pos: {self.pos} ')
        #print(f'clase: {a.clase}, vc: {a.v_c}, w: {a.wealth}, pos: {a.pos}' )    
    
    def balance(self):
        
        if self.wealth < 0:
        #    self.wealth = 0 # Se perdonan las deudas
           self.clase = -1
        if 0 <= self.wealth <= self.model.umbral_M:
            self.clase = -1
        elif self.model.umbral_M < self.wealth <= self.model.umbral_A:
            self.clase = 0
        else:
            self.clase = 1
        #print(f'clase actualizada: {self.clase}, vc: {self.v_c}, wealth: {self.wealth}, pos: {self.pos} ')      
        
        
#---------------------- STEP METHOD ---------------------------------
    #contador=0
    def step(self):
    #ojo con el orden de activación        
        self.agent_spend()
        #self.balance()
        self.vc_update()
        
        #print('End of step')
#         self.contador += 1 #el contador es para introducir los volados cada "j" iteraciones
#         if self.contador%1 == 0: 
#             self.proba()
        
        
    

In [3]:

#----------------- funciones para graficar en la simulacion visual--------------------------
def bajos(model):
    cont = 0
    for a in model.schedule.agents:
        if a.clase == -1:
            cont += 1
            #cont += a.wealth
    return cont

def medios(model):
    cont = 0
    for a in model.schedule.agents:
        if a.clase == 0:
            cont += 1
            #cont += a.wealth
    return cont

def altos(model):
    cont = 0
    for a in model.schedule.agents:
        if a.clase == 1:
            cont += 1
            #cont += a.wealth
    return cont

def bajos_w(model):
    cont = 0
    arreglo = []
    for a in model.schedule.agents:
        if a.clase == -1:
            cont += a.wealth
            arreglo.append(a.wealth)
    #print("riqueza pobre")
    #print(arreglo)
    #print("---"*6)
    return cont

def medios_w(model):
    cont = 0
    for a in model.schedule.agents:
        if a.clase == 0:           
            cont += a.wealth
    return cont

def altos_w(model):
    cont = 0
    for a in model.schedule.agents:
        if a.clase == 1:
            cont += a.wealth
    return cont

def riqueza(model):
    cont=0
    for a in model.schedule.agents:
        cont += a.wealth
    return cont

In [4]:
class KeynesModel(Model):
    
    """Model class for the Postkeynessian consumer model"""
    def __init__(self, N, m, n, n_b, n_m, n_a):
        """
        N = numero de agentes
        m x n = numero total de celdas del modelo
            m = numero de filas
            n = numero de columnas
        *args = lista/tupla (??) con entradas n_b , n_m , n_a, donde
            n_b = numero de agentes de clase baja
            n_m = numero de agentes de clase media
            n_a = numero de agentes de clase alta
        """
        args=(n_b,n_m,n_a)
        
        if np.array(args).sum() != N:
            print("Error, la suma de agentes de cada clase no es igual a N")
        else:
            
            self.num_agents = N
            
            #umbrales de riqueza (puedo pedir los parametros: modificar los argumentos del init)
            self.umbral_M = 7
            self.umbral_A = 33
            self.wealth_MAX = 36
            #
            self.grid = SingleGrid(m, n, torus=True)
            self.schedule = RandomActivation(self) 
            self.running = True
#             self.count = 0
#             self.count_cord = []
        
            #CREATE AGENTS
            # matrix of all coordinates, ramdom choosing from this to set agent's position and 
            # then  must asign V_c from a list of numpy arrays
            # order: clase baja, clase media, clase alta
            
            M=[(i,j) for i in range(m) for j in range(n)] #matriz de posiciones
            n_b, n_m, n_a=args[0], args[1], args[2] #linea innsecesaria
            for j in range(3):
                v_c=[0,0,0]
                for i in range(args[j]):
                    v_c[j]=1
                    #x = self.random.randrange(m)
                    #y = self.random.randrange(n)
                    #pos=(x,y)
                    pos=M.pop(M.index(rm.choice(M)))
                    #print(pos)
                    v_c=np.array(v_c)
                    
                    # random wealth generator
                    #comenzamos asignando con distribuciones uniformes
                    if j == 0:
                        w = rm.choice(range(self.umbral_M+1))
                    elif j == 1:
                        w = rm.choice(range(self.umbral_M+1,self.umbral_A+1))
                    else:
                        w = rm.choice(range(self.umbral_A+1,self.wealth_MAX+1))
                    #
                    a=KeynesAgent(self, pos, v_c, w)
                    a.clase = j-1
                    self.schedule.add(a)
                    self.grid.position_agent(a,pos) 
                    #print(f'clase: {a.clase}, vc: {a.v_c}, w: {a.wealth}, pos: {a.pos}' )
        self.datacollector = DataCollector(model_reporters=
                                           {"BAJOS": bajos, "MEDIOS": medios, "ALTOS": altos,"w_total":riqueza,
                                            "BAJOS_W": bajos_w, "MEDIOS_W": medios_w, "ALTOS_W": altos_w})
                    
    def step(self):
        self.datacollector.collect(self)
        self.schedule.step()
        
        

In [5]:
N, m, n, nb, nm, na = 100,10,10,50,25,25
model = KeynesModel(N, m, n, nb, nm, na)

In [6]:
def agent_portrayal(agent):
    portrayal = {"Shape": "circle",
                 "Filled": "true",
                 "Layer": 0,
                 "r": 0.5}
    
    D=dict(B=np.array([1,0,0]), M=np.array([0,1,0]), A=np.array([0,0,1]),
       MB=np.array([1,1,0]), AB=np.array([1,0,1]), AM=np.array([0,1,1]),
       AMB=np.array([1,1,1]))
    
    if (agent.v_c == D['B']).all() :
        portrayal["Color"] = "red"        
    elif (agent.v_c == D['M']).all():
        portrayal["Color"] = "green" 
    elif (agent.v_c == D['A']).all():
        portrayal["Color"] = "blue"
    elif (agent.v_c == D['MB']).all():
        portrayal["Color"] = "yellow"
    elif (agent.v_c == D['AB']).all():
        portrayal["Color"] = "#778899"#"#FF1493" #Gris
    elif (agent.v_c == D['AM']).all():
        portrayal["Color"] = "#48D1CC" #turquesa bonito
    elif (agent.v_c == D['AMB']).all():
        portrayal["Color"] = "#000000" #negro
    else :
        portrayal["Color"] = "#FFFFFF" #blanco
        
      
    return portrayal

In [7]:
def agent_portrayal2(agent):
    portrayal = {"Shape": "circle",
                 "Filled": "true",
                 "Layer": 0,
                 "r": 0.5}
    
#     D=dict(B=np.array([1,0,0]), M=np.array([0,1,0]), A=np.array([0,0,1]),
#        MB=np.array([1,1,0]), AB=np.array([1,0,1]), AM=np.array([0,1,1]),
#        AMB=np.array([1,1,1]))
    
    if agent.clase == -1 :
        portrayal["Color"] = "red"
        
    elif agent.clase == 0:
        portrayal["Color"] = "green" 
#     else:
#         portrayal["Color"] = "blue"
    else:
        portrayal["Color"] = "blue"
        
#     elif (agent.v_c == D['MB']).all():
#         portrayal["Color"] = "yellow"
        
#     elif (agent.v_c == D['AB']).all():
#         portrayal["Color"] = "#778899"#"#FF1493" #rosa mexicano
        
#     elif (agent.v_c == D['AM']).all():
#         portrayal["Color"] = "#48D1CC" #turquesa bonito
        
#     elif (agent.v_c == D['AMB']).all():
#         portrayal["Color"] = "#000000" #negro
#     else :
#         portrayal["Color"] = "#FFFFFF" #blanco
        
      
    return portrayal

In [8]:
bj = {"Label": "BAJOS", "Color": "Red"}
md = {"Label": "MEDIOS", "Color": "Green"}
lt = {"Label": "ALTOS", "Color": "Blue"}
chart = ChartModule([bj, md, lt],
                    data_collector_name='datacollector')
bj_w = {"Label": "BAJOS_W", "Color": "Red"}
md_w = {"Label": "MEDIOS_W", "Color": "Green"}
lt_w = {"Label": "ALTOS_W", "Color": "Blue"}
chart_w = ChartModule([bj_w, md_w, lt_w],
                    data_collector_name='datacollector')
w_t = {"Label": "w_total", "Color": "Black" }
chart2 = ChartModule([w_t],data_collector_name="datacollector")

# Visualización y análisis de datos

In [9]:
#parametros del modelo
N, m, n, nb, nm, na = 100,10,10,0,0,100

grid = CanvasGrid(agent_portrayal, m, n, 500, 500)
grid_2 = CanvasGrid(agent_portrayal2, m, n, 500, 500)
server = ModularServer(KeynesModel,
                       [grid,grid_2,chart,chart2, chart_w],
                       "Post-Keynesian Model",
                       {"N":N, "m":m, "n":n, "n_b":nb, "n_m":nm, "n_a":na})
server.port = 1175# The default
server.launch()

Interface starting at http://127.0.0.1:1175


RuntimeError: This event loop is already running

In [ ]:
N, m, n, nb, nm, na = 100,10,10,33,34,33
model = KeynesModel(N, m, n, nb, nm, na)

In [ ]:
agent_wealth = [a.wealth for a in model.schedule.agents] #list comprehension :)
plt.hist(agent_wealth, bins=range(max(agent_wealth)+1))
plt.title('Distribución inicial'), plt.xlabel('Riqueza'), plt.ylabel('N agentes')
#For a script add the following line
plt.show()
print(f'max: {max(agent_wealth)}, mean:{np.array(agent_wealth).mean()}, std: {round(np.array(agent_wealth).std(),2)}')

In [ ]:
for i in range(101):
    model.step()
    if i%50==0 or i==10:
        agent_wealth = [a.wealth for a in model.schedule.agents] #list comprehension :)
        plt.hist(agent_wealth, bins=range(max(agent_wealth)+1))
        plt.title(f'iter: {i}'), plt.xlabel('Riqueza'), plt.ylabel('N agentes')
    #For a script add the following line
        plt.show()
        print(f'max: {max(agent_wealth)}, mean:{np.array(agent_wealth).mean()}, std: {round(np.array(agent_wealth).std(),2)}')
        input('type Enter to continue')

Socket opened!
{"type":"reset"}
{"type":"get_step","step":1}
{"type":"get_step","step":2}
{"type":"get_step","step":3}
{"type":"get_step","step":4}
{"type":"get_step","step":5}
{"type":"get_step","step":6}
{"type":"get_step","step":7}
{"type":"get_step","step":8}
{"type":"get_step","step":9}
{"type":"get_step","step":10}
{"type":"get_step","step":11}
{"type":"get_step","step":12}
{"type":"get_step","step":13}
{"type":"get_step","step":14}
{"type":"get_step","step":15}
{"type":"get_step","step":16}
{"type":"get_step","step":17}
{"type":"get_step","step":18}
{"type":"get_step","step":19}
{"type":"get_step","step":20}
{"type":"get_step","step":21}
{"type":"get_step","step":22}
{"type":"get_step","step":23}
{"type":"get_step","step":24}
{"type":"get_step","step":25}
{"type":"get_step","step":26}
{"type":"get_step","step":27}
{"type":"get_step","step":28}
{"type":"get_step","step":29}
{"type":"get_step","step":30}
{"type":"get_step","step":31}
{"type":"get_step","step":32}
{"type":"get_step

# Discusión/ Preguntas y Respuestas

* La distribución final del número de agentes en cada categoría **siempre** tiene mas agentes de clase baja, muy pocos ricos y una cantidad "moderada" de agentes de clase media


* La distribución de la riqueza converge siempre a una distribución descrita como una ley de potencias, lo cual se parece a los resultados de la econofísica estudiados en 2003 en [Dragulesco]. Entender el origen de este fenómeno es muy importante desde la perespectiva eonómica-política


Como *perespectivas a futuro*:


* La implementación de otras reglas de "gasto" y selección de provedores


* Introducir una variable de desarrollo tecnológico ("emprendedores") que permitan el aumento de la riqueza total del sistema (en términos termodinámicos, introducir energía al sistema

* Estudio de variables de tipo impuesto

* Implementación de la idea del "Universal Basic Income (UBI)" 

Creemos que el objetivo de implemtar estas y otras ideas es tratar de cambiar la forma de la distribucion de la riqueza o al menos lograr un dezplazamiento positivo en el eje $x$.


# $$  \mathrm{¡ Gracias \ por \ su \ atención !} $$


# Referencias

* [Vite-Carreón, 2015] - Vite, R. \& Carreón, G. (2015). Actas de Economía y Complejidad. Ciudad Universitaria, CDMX: CEIICH-UNAM.

* [Dragulescu, 2003] - Dragulescu, A \& Yakovenko, V.. (2003). Statistical Mechanics of Money, Income, and Wealth: A Short Survey. AIP Conference Proceedings, 661, 180-183. 

* [Mesa] - Mesa: Python alternative for agent based modelling: [Read the docs](https://mesa.readthedocs.io/en/masterindex.html), [GitHub](https://github.com/projectmesa/mesa), [Tutorials](https://mesa.readthedocs.io/en/master/tutorials/intro_tutorial.html)

